## Export Compliance TSV from the FairNow APIs

#### This generates a TSV export of the Compliance report, containing applications, and frameworks.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

## Create Compliance DataFrame
#### Graphs can be created using this DataFrame output

In [ ]:
from utils.fairnow import export_to_tsv, create_compliance_df

compliance_df = create_compliance_df(client_id='2726p9mj2ghit00n23tkjatp84') # Replace with your Client Id

### Export to TSV

In [ ]:
export_to_tsv(compliance_df, 'compliance_export.tsv')